In [ ]:
import pandas as pd
import json
from datetime import timedelta, datetime

# 数据路径和存储路径

In [ ]:
data1_path = "data/train.csv"
data2_path = "data/test.csv"
data1_to_csv_path = "data/train_clean.csv"
data2_to_csv_path = "data/test_clean.csv"
json_path = "data/index_json.json"

json_dict = {}

In [ ]:
data1 = pd.read_csv(data1_path)
data2 = pd.read_csv(data2_path)

# 删除部分特征

In [ ]:
def del_features(data):
    del_list = ["sid", "ver", "province", "idfamd5", "make", "os", "reqrealip"]
    for i in del_list:
        del data[i]

    print("del ok")

In [ ]:
del_features(data1)
del_features(data2)

# 连续值特征处理

In [ ]:
def nginxtime(data):
	data['datetime'] = pd.to_datetime(data['nginxtime'] / 1000, unit='s') + \
					   timedelta(hours=8)
	data['hour'] = data['datetime'].dt.hour
	data['day'] = data['datetime'].dt.day - data['datetime'].dt.day.min()
	data['minute'] = data['datetime'].dt.minute.astype('uint8')
	data["time"] = data['day'] * 24 * 60 + data['hour'] * 60 + data['minute']

	for i in ["datetime", "hour", "day", "minute"]:
		del data[i]

	print("nginxtime ok")

In [ ]:
nginxtime(data1)
nginxtime(data2)

In [ ]:
def h_w_ppi(data):
	new_h = data["h"] + 0.1
	new_w = data["w"] + 0.1
	new_ppi = data["ppi"] + 0.1
	resolution_ratio = new_h * new_w * new_ppi

	del data["h"]
	del data["w"]
	del data["ppi"]

	data["resolution_ratio"] = resolution_ratio
	print("h_w_ppi ok")

In [ ]:
h_w_ppi(data1)
h_w_ppi(data2)

In [ ]:
def ip(data):
	# 从网上爬取的ip对应城市的json文件导入
	with open("data/ip_index.json", "r") as f:
		ip_index = json.load(f)

	new_ip = []
	for n, i in enumerate(data["ip"]):
		try:
			if ip_index[i] in data["city"][n]:
				new_ip.append(0)
			else:
				new_ip.append(1)
		except:
			new_ip.append(1)

	data["ip"] = new_ip
	print("ip ok")

In [ ]:
ip(data1)
ip(data2)

# one-hot特征处理

In [ ]:
def dvctype(data):
	new_dvctype = []
	for i in data["dvctype"]:
		if i == 3:
			i = 1
		new_dvctype.append(i)

	data["dvctype"] = new_dvctype
	print("dvctype ok")

In [ ]:
dvctype(data1)
dvctype(data2)

In [ ]:
def orientation(data):
	new_orientation = []
	for i in data["orientation"]:
		if i == 90:
			i = 2
		new_orientation.append(i)

	data["orientation"] = new_orientation
	print("orientation ok")

In [ ]:
orientation(data1)
orientation(data2)

In [ ]:
def apptype(data1, data2):
	global json_dict

	def process(data):
		new_apptype = []
		for i in data["apptype"]:
			try: 
				new_apptype.append(apptype_set.index(i))
			except:
				new_apptype.append(len(apptype_set))
		data["apptype"] = new_apptype

	apptype_set = list(set(data1["apptype"]) & set(data2["apptype"]))
	json_dict["apptype"] = apptype_set

	process(data1)
	process(data2)
	print("apptype ok")

In [ ]:
apptype(data1, data2)

In [ ]:
def carrier(data1, data2):
	global json_dict

	def process(data):
		new_carrier = []
		for i in data["carrier"]:
			new_carrier.append(carrier_set.index(i))
		data["carrier"] = new_carrier

	carrier_set = list(set(data1["carrier"]) & set(data2["carrier"]))
	json_dict["carrier"] = carrier_set

	process(data1)
	process(data2)
	print("carrier ok")

In [ ]:
carrier(data1, data2)

In [ ]:
def lan(data1, data2):
	global json_dict

	def process_1(data):
		new_lan = []
		for i in data["lan"]:
			try:
				for j in i:
					if j in lan_del_string:
						i = i.replace(j,"")

				i = i.lower()
			except:
				pass
			finally:
				new_lan.append(i)

		return new_lan

	def process_2(data, new_lan):
		for n, i in enumerate(new_lan):
			try:
				new_lan[n] = lan_set.index(i)
			except:
				new_lan[n] = len(lan_set)
		data["lan"] = new_lan

	lan_del_string = ["_", "-"]

	new_lan_1 = process_1(data1)
	new_lan_2 = process_1(data2)

	lan_set = list(set(new_lan_1) & set(new_lan_2))
	json_dict["lan"] = lan_set

	process_2(data1, new_lan_1)
	process_2(data2, new_lan_2)
	print("lan ok")

In [ ]:
lan(data1, data2)

# embedding特征数据预处理

In [ ]:
def model(data):
	model_del_string = [" ", "_", ",", "+", "/", "-", "%", "(", ")", "."]
	special_type = {'PACM00': "OPPO R15", 'PBAM00': "OPPO A5", \
					'PBEM00': "OPPO R17", 'PADM00': "OPPO A3", \
					'PBBM00': "OPPO A7", 'PAAM00': "OPPO R15_1", \
					'PACT00': "OPPO R15_2", 'PABT00': "OPPO A5_1", \
					'PBCM10': "OPPO R15x"}
	new_model = []

	# 把一些定制手机的型号转换成一般形式
	for i in special_type:
		data['model'].replace(i, special_type[i], inplace=True)

	for i in data["model"]:
		try:
			for j in i:
				if j in model_del_string:
					i = i.replace(j,"")

			i = i.lower()
			if "huaweihuawei" in i:
				i = i.replace("huaweihuawei","huawei")
			if "xiaomixiaomi" in i:
				i = i.replace("xiaomixiaomi","xiaomi")
		except:
			pass
		finally:
			new_model.append(i)

	data["model"] = new_model
	print("model ok")

In [ ]:
model(data1)
model(data2)

In [ ]:
def osv(data):
	new_osv = []
	for i in data["osv"]:
		try:
			if "," in i:
				i = i.replace(",", ".")
			if " 十核2.0G_HD" in i:
				i = i.replace(" 十核2.0G_HD", "")

			process = i.split(".")
			while process[-1] == "0":
				del process[-1]
			i = ".".join(process)
		except:
			pass
		finally:
			new_osv.append(i)

	data["osv"] = new_osv
	print("osv ok")

In [ ]:
osv(data1)
osv(data2)

# embedding特征处理

In [ ]:
def embedding(data1, data2):
	global json_dict

	def build_index(n, data, feature):
		new_feature = []
		for i in data[feature]:
			try:
				new_feature.append(embedding_set.index(i))
			except:
				new_feature.append(len(embedding_set) + n)

		return new_feature

	embedding_index = ["pkgname", "adunitshowid", "mediashowid", "city", \
					   "adidmd5", "imeimd5","openudidmd5", "macmd5", \
					   "model", "osv"]
	embedding_set = []

	for i in embedding_index:
		embedding_set += list(set(data1[i]) & set(data2[i]))

	embedding_set = list(set(embedding_set))
	json_dict["embedding"] = embedding_set

	for n, i in enumerate(embedding_index):
		data1[i] = build_index(n, data1, i)
		data2[i] = build_index(n, data2, i)
		print("{} is ok".format(i))

	print("embedding ok")

In [ ]:
embedding(data1, data2)

# 交换列顺序

In [ ]:
def change_col_index(data):
	col_index = ["label", "time", "ip", "resolution_ratio", "apptype", \
				"dvctype", "ntt", "carrier", "orientation", "lan", "pkgname", \
				"adunitshowid", "mediashowid", "city", "adidmd5", \
				"imeimd5","openudidmd5", "macmd5", "model", "osv"]
	if data is data2:
		del col_index[0]

	return data[col_index]

In [ ]:
change_col_index(data1)
change_col_index(data2)

# 保存

In [ ]:
data1.to_csv(data1_to_csv_path, index=False)
data2.to_csv(data2_to_csv_path, index=False)
with open(json_path,"w") as f:
    json.dump(json_dict, f)